# HW5 - Logistic Regression & SVM
##### 負責助教：張逸群 toosyou.tw@gmail.com

> 這次的作業將帶大家使用 Logistic Regression 跟 SVM 去做乳癌是否是良性的預測

##### 請加入 Discord 群組，以獲得助教線上協助：https://discord.gg/Ug9VGSt

---
## 評分標準
* 上課繳交：25%
    * 在當天 16:20 前上傳至 new E3 作業，請將以下打包成 .zip 上傳：
        * 程式原始檔 (.ipynb）
        * 程式 .html 檔
* 程式分數：75%
    * 於一週內完成並上傳至 new E3
    
#### 注意：最終學期成績將視全班原始成績做調整

## 資料簡介
* Data - Breast cancer wisconsin (diagnostic) dataset
    * Number of Instances: 569
    * Number of Attributes: 30 numeric, predictive attributes and the class
    * Attribute Information:
        * radius (mean of distances from center to points on the perimeter)
        * texture (standard deviation of gray-scale values)
        * perimeter
        * area
        * smoothness (local variation in radius lengths)
        * compactness (perimeter^2 / area - 1.0)
        * concavity (severity of concave portions of the contour)
        * concave points (number of concave portions of the contour)
        * symmetry
        * fractal dimension (“coastline approximation” - 1)
        * The mean, standard error, and “worst” or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features. For instance, field 3 is Mean Radius, field 13 is Radius SE, field 23 is Worst Radius.
        * **class**:
            * WDBC-Malignant
            * WDBC-Benign
        
---

#### 姓名：陳泊均
#### 學號：0616043

## Packages that Might Be Used

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import matplotlib.pyplot as plt
%matplotlib inline

## Load Data - 20%

#### 請使用 `pd.read_csv` 將 `data.csv` 讀入成 `DataFrame` 至 `df` - 10%

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


#### 請將除了 `id` 和 `diagnosis` 外的 column 指派為 `X`（此為我們使用的 feature），並將 `diagnosis` column 指派為 `y`（此為我們將預測的目標）- 10%

In [4]:
X = df.iloc[:,2:-1]
y = df['diagnosis']

In [5]:
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [6]:
y.head()

0    M
1    M
2    M
3    M
4    M
Name: diagnosis, dtype: object

## Train-Test Split - 10%

#### 請使用 `train_test_split` 將 `X` 與 `y` 分成 Training Set 以及 Testing Set - 10%

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## Logistic Regression - 30%

#### 請使用 [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 及 **`Training Set`** 對 [LogisticRegression Model](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 進行 [Hyperparamter Search](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator) - 10%

> Model 的挑選嚴格禁止使用 testing set，否則無法保證挑出來的 model 對於新的資料是否還是適用

* 固定參數
    * tol: 0.01
    * penalty： l2
* Search 範圍
    * C: 1, 100
    * solver: saga lbfgs
    * warm_start: True, False
    

In [8]:
# turn off convergence warning 'cause it's annoying
import warnings
from sklearn.exceptions import ConvergenceWarning

warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [9]:
parameters = {'C':[1, 100], 'solver':['saga', 'lbfgs'], 'warm_start':[True, False]}
param = {'C':[1, 100], 'penalty':['l2'], 'solver':['saga', 'lbfgs'], tol=[0.01] 'warm_start':[True, False]}
lr_grid = GridSearchCV(LogisticRegression(tol=0.01, penalty='l2'), param_grid=parameters, cv=5, return_train_score=False)
lr_grid.fit(X_train, y_train)

C:\Users\ilike\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.01, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [1, 100], 'solver': ['saga', 'lbfgs'], 'warm_start': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

![meme1](https://imgur.com/QzAOB3V.jpg)

#### 將 Search 結果印出 - 0%

In [10]:
pd.DataFrame(lr_grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,param_warm_start,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
6,0.017752,0.001163,0.001405,4.981928e-04,100,lbfgs,True,"{'C': 100, 'solver': 'lbfgs', 'warm_start': True}",0.9625,0.9375,0.9750,0.9625,0.897436,0.947236,0.027454,1
7,0.017944,0.001772,0.000998,1.059824e-06,100,lbfgs,False,"{'C': 100, 'solver': 'lbfgs', 'warm_start': Fa...",0.9625,0.9375,0.9750,0.9625,0.897436,0.947236,0.027454,1
2,0.020440,0.002821,0.000798,3.989460e-04,1,lbfgs,True,"{'C': 1, 'solver': 'lbfgs', 'warm_start': True}",0.9625,0.9125,0.9625,0.9625,0.897436,0.939698,0.028500,3
3,0.019250,0.001395,0.001197,3.985167e-04,1,lbfgs,False,"{'C': 1, 'solver': 'lbfgs', 'warm_start': False}",0.9625,0.9125,0.9625,0.9625,0.897436,0.939698,0.028500,3
0,0.025131,0.012918,0.002993,3.988576e-03,1,saga,True,"{'C': 1, 'solver': 'saga', 'warm_start': True}",0.9250,0.8625,0.8875,0.9000,0.884615,0.891960,0.020529,5
1,0.014959,0.000631,0.000998,3.234067e-07,1,saga,False,"{'C': 1, 'solver': 'saga', 'warm_start': False}",0.9250,0.8625,0.8875,0.9000,0.884615,0.891960,0.020529,5
4,0.018237,0.002610,0.000999,4.672031e-07,100,saga,True,"{'C': 100, 'solver': 'saga', 'warm_start': True}",0.9250,0.8625,0.8875,0.9000,0.884615,0.891960,0.020529,5
5,0.016954,0.002523,0.001397,4.883442e-04,100,saga,False,"{'C': 100, 'solver': 'saga', 'warm_start': False}",0.9250,0.8625,0.8875,0.9000,0.884615,0.891960,0.020529,5


#### 請回答以下問題 - 10%

1. 請問 Search 結果中，表現最好的是哪一組參數？ - 5%
    * {C: 100, solver: lbfgs, warm_start: True}
    * {C: 100, solver: lbfgs, warm_start: False}
2. 承上，請問是以何作為依據？ - 5%
    * mean_test_score (lr_grid.best_score_)

#### 自 `lr_grid` 中取出最好的模型，並且使用完整的 **Training Set** 重新訓練，最後使用 **Testing Set** 計算 Accuracy - 10%

In [11]:
best_lr_clf = GridSearchCV(LogisticRegression(tol=0.01, penalty='l2'), \
                           param_grid={'C':[100], 'solver':['lbfgs'], 'warm_start':[True]}, cv=5, \
                           return_train_score=False)
best_lr_clf.fit(X_train, y_train)

print(best_lr_clf)
print('lr acc:', best_lr_clf.best_score_)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.01, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [100], 'solver': ['lbfgs'], 'warm_start': [True]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)
lr acc: 0.9472361809045227


C:\Users\ilike\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


## SVM - 30%

#### 請使用 [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) 及 **`Training Set`** 對 [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) 進行 [Hyperparamter Search](https://scikit-learn.org/stable/modules/grid_search.html#tuning-the-hyper-parameters-of-an-estimator) - 10%
* Search 範圍
    * kernel： linear, rbf
    * C: 1, 10

In [12]:
parameters = {'kernel': ['linear', 'rbf'], 'C': [1, 10]}
svc_grid = GridSearchCV(SVC(gamma='scale'), param_grid=parameters, cv=5, return_train_score=False)
svc_grid.fit(X_train, y_train)

C:\Users\ilike\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'kernel': ['linear', 'rbf'], 'C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)

#### 將 Search 結果印出 - 0%

In [13]:
pd.DataFrame(svc_grid.cv_results_).sort_values('rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
2,3.071157,2.285351e+00,0.001398,0.000488,10,linear,"{'C': 10, 'kernel': 'linear'}",0.9750,0.9625,0.975,0.9875,0.923077,0.964824,0.022082,1
0,1.264184,5.319621e-01,0.001005,0.000012,1,linear,"{'C': 1, 'kernel': 'linear'}",0.9625,0.9250,1.000,0.9625,0.910256,0.952261,0.031550,2
3,0.005977,6.315101e-04,0.001198,0.000399,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.9625,0.9375,0.925,0.9250,0.923077,0.934673,0.014874,3
1,0.004986,2.861023e-07,0.001197,0.000399,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.9375,0.9125,0.925,0.9125,0.910256,0.919598,0.010375,4


#### 請回答以下問題 - 10%

1. 請問 Search 結果中，表現最好的是哪一組參數？ - 5%
    * {C: 10, kernel: linear}
2. 承上，請問是以何作為依據？ - 5%
    * mean_test_score (svc_grid.best_score_)

![出作業](https://imgur.com/qmMHznz.jpg)

#### 自 `svc_grid` 中取出最好的模型，並且使用完整的 **Training Set** 重新訓練，最後使用 **Testing Set** 計算 Accuracy - 10%

In [14]:
best_svc_clf = GridSearchCV(SVC(gamma='scale'), param_grid={'C':[10], 'kernel':['linear']}, \
                            cv=5, return_train_score=False)
best_svc_clf.fit(X_train, y_train)

print(best_svc_clf)
print('svc acc:', best_svc_clf.best_score_)

C:\Users\ilike\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [10], 'kernel': ['linear']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring=None, verbose=0)
svc acc: 0.964824120603015


## Prediction - 10%

#### 假設現在有一個新的病人資料如下，請使用上面建立的 `best_lr_clf` 以及 `best_svc_clf` 分別對他進行預測，並印出預測結果 - 10%

In [15]:
X_new = [11.61,
 16.02,
 75.46,
 408.2,
 0.1088,
 0.1168,
 0.07097,
 0.044969999999999996,
 0.1886,
 0.0632,
 0.2456,
 0.7339,
 1.6669999999999998,
 15.89,
 0.005884,
 0.02005,
 0.026310000000000004,
 0.013040000000000001,
 0.01848,
 0.001982,
 12.64,
 19.67,
 81.93,
 475.7,
 0.1415,
 0.217,
 0.2302,
 0.1105,
 0.2787,
 0.07427]

In [16]:
X_new = np.array(X_new).reshape(1, -1)
lr_pred = best_lr_clf.predict(X_new)
svc_pred = best_svc_clf.predict(X_new)

In [17]:
print('lr prediction:', lr_pred)
print('svc prediction:', svc_pred)

lr prediction: ['B']
svc prediction: ['B']


![ml_model](https://imgur.com/JsbckWU.jpg)

# 歡迎跟 ~~平常沒有人可以說話的~~ 助教縮縮話 <3

## 跪求助教大大梗圖網址 ^_^😯